In [2]:
import numpy as np
def pf(func):
    def printfunc(*args, **kwargs):
        print(func, args, kwargs)
        return func(*args, **kwargs)
    
    return printfunc

class A:
    @pf
    def __init__(self, name, num_qubits, num_clbits, params, duration=None, unit='dt'):
        self.name = name
        self.num_qubits = num_qubits
        self.num_clbits = num_clbits
        self.params = params
        self.duration = duration
        self.unit = unit
        print(self.bar())
        
    def foo():
        return "foo A"
    
class B(A):
    @pf
    def __init__(self, name, qudit_dimensions, num_single_qudits, num_clbits, params, duration=None, unit='dt'):
        self.num_single_qudits = num_single_qudits
        self.qudit_dimensions = qudit_dimensions
        A.__init__(self, name, int(sum(np.ceil(np.log2(dimension)) for dimension in qudit_dimensions))+num_single_qudits, num_clbits, params, duration=duration, unit=unit)
        
    def bar(self):
        return "bar BA"
    
    def doe(self):
        return "doe BA"

class C(A):
    @pf
    def __init__(self, name, num_qubits, params, label):
        self.label = label
        super().__init__(name, num_qubits, 0, params)
        
    __array_priority__ = 20
        
    def foo(self):
        return "foo CA"
    
    def doe(self):
        return "doe CA"

class D(B, C):
    @pf
    def __init__(self, name, qudit_dimensions, num_single_qudits, params, label):
        self.label = label
        B.__init__(self, name, qudit_dimensions, num_single_qudits, 0, params)
        C.__init__(self, name, int(sum(np.ceil(np.log2(dimension)) for dimension in qudit_dimensions)) +num_single_qudits, params, label)
        
    def bar(self):
        return "bar DBC"

print(D.mro())
d = D("qft", [2,3,5], 2, "t", "QFT")
#print(d.name, d.label, d.qudit_dimensions, d.num_qubits, d.num_clbits, d.params, d.duration, d.unit)
#d.foo(), d.bar(), d.doe()
#d.__array_priority__
#issubclass(B, (D,A))

[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]
<function D.__init__ at 0x000002BAC3C2AE50> (<__main__.D object at 0x000002BAC284C220>, 'qft', [2, 3, 5], 2, 't', 'QFT') {}
<function B.__init__ at 0x000002BAC3C2A9D0> (<__main__.D object at 0x000002BAC284C220>, 'qft', [2, 3, 5], 2, 0, 't') {}
<function A.__init__ at 0x000002BAC3C2A700> (<__main__.D object at 0x000002BAC284C220>, 'qft', 8, 0, 't') {'duration': None, 'unit': 'dt'}
bar DBC
<function C.__init__ at 0x000002BAC3C2AC10> (<__main__.D object at 0x000002BAC284C220>, 'qft', 8, 't', 'QFT') {}
<function A.__init__ at 0x000002BAC3C2A700> (<__main__.D object at 0x000002BAC284C220>, 'qft', 8, 0, 't') {}
bar DBC


In [91]:
from itertools import product, zip_longest

class Qudit:
    def __init__(self, d):
        self.d = d
        
    def __repr__(self):
        return "Qd"+str(self.d)+" "+str(id(self))

class Qubit:
    def __repr__(self):
        return "Qb_ "+str(id(self))

qudit_dimensions = [4,3,9]
num_single_qubits = 2

def qd_broadcast_arguments(qdargs, qargs, cargs):
    print(qdargs, qargs)
   
    if all(len(qdarg) == 1 for qdarg in qdargs) and [qdarg[0].d for qdarg in qdargs] == qudit_dimensions:
        qdargs = [[qdarg[0] for qdarg in qdargs]]
    elif not (len(qdargs) != 0 and all([qd.d for qd in qdarg] == qudit_dimensions for qdarg in qdargs)):
        raise ValueError("qdargs")
    
    if all(len(qarg) == 1 for qarg in qargs) and len(qargs) == num_single_qubits:
        qargs = [[qarg[0] for qarg in qargs]]
    elif not (len(qargs) != 0 and all(len(qarg) == num_single_qubits for qarg in qargs)):
        raise ValueError("qargs")
    
    if cargs:
        raise ValueError("cargs")
    
    print(qdargs, qargs)
    for qdarg, qarg in product(qdargs, qargs):
        yield qdarg, qarg, []
    

list(qd_broadcast_arguments([[Qudit(4), Qudit(3), Qudit(9)], [Qudit(4), Qudit(3), Qudit(9)]],[[Qubit()], [Qubit()]],[]))

[[Qd4 1861940351520, Qd3 1861940354112, Qd9 1861940351280], [Qd4 1861940351232, Qd3 1861940354256, Qd9 1861940353104]] [[Qb_ 1861940353632], [Qb_ 1861940351136]]
[[Qd4 1861940351520, Qd3 1861940354112, Qd9 1861940351280], [Qd4 1861940351232, Qd3 1861940354256, Qd9 1861940353104]] [[Qb_ 1861940353632, Qb_ 1861940351136]]


[([Qd4 1861940351520, Qd3 1861940354112, Qd9 1861940351280],
  [Qb_ 1861940353632, Qb_ 1861940351136],
  []),
 ([Qd4 1861940351232, Qd3 1861940354256, Qd9 1861940353104],
  [Qb_ 1861940353632, Qb_ 1861940351136],
  [])]

In [26]:
qd = 2
q = 3
c = 1
qdarg = "1234"
qarg = "abcdef"
carg = "qr"

qdit = zip(*([iter(qdarg)]*2))
qit = zip(*([iter(qarg)]*3))
cit = zip(*([iter(carg)]*1))

for x,y,z in zip(qdit, qit, cit):
    print(list(x), list(y), list(z))

['1', '2'] ['a', 'b', 'c'] ['q']
['3', '4'] ['d', 'e', 'f'] ['r']


In [1]:
qargs = 1
qargs = qargs if qargs else []
qargs

1

In [39]:
class X():    
    def __init__(self):
        self._data = []
        
    @property
    def data(self):
        print("get data X")
        return self._data
    
    @data.setter
    def data(self, data):
        print("set data X")
        self._data = data
    
    def foo(self):
        print(self._data)
        print(self.data)
        

class Y(X):
    
    def __init__(self):
        super().__init__()
    
    @X.data.getter
    def data(self):
        print("get data Y")
        return self._data, None

    def bar(self):
        return self.data
        
y = Y()
print(y.data,"\n")
y._data.extend([1])
y.foo()
y.bar()
y.data= [100]
y.data

get data Y
([], None) 

[1]
get data Y
([1], None)
get data Y
set data X
get data Y


([100], None)

In [146]:
from itertools import zip_longest

def foo(qdargs, qargs, cargs):
    qdit = zip(*([iter(qdargs)] * 2))
    qit = zip(*([iter(qargs)] * 1))
    cit = zip(*([iter(cargs)] * 1))
    print([x for x in cit])
    cit = [()]*4
    print(cit)
    for qds, qs, cs in zip(qdit, qit, cit):
        yield list(qds), list(qs), list(cs)
        
[x for x in foo([0,3,21,123], [3,2], [2,3])]

x= 1 if 34 else 2
x

[(2,), (3,)]
[(), (), (), ()]


1

In [66]:
def foo(*args):
    print(args)
    
x=[1,2]
foo(*iter(x))

(1, 2)
